In [ ]:
!pip install ultralytics

In [ ]:
import kagglehub

path = kagglehub.dataset_download("neelpratiksha/indian-traffic-sign-dataset")
print("Path to dataset files:", path)

100%|██████████| 25.8M/25.8M [00:00<00:00, 30.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1


In [ ]:
import os
import pandas as pd

dataset_dir = "/root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1/Indian-Traffic Sign-Dataset/Images"
files = os.listdir(dataset_dir)
print("Dataset contents:", files)

Dataset contents: ['21', '36', '7', '10', '11', '24', '3', '42', '2', '0', '8', '26', '35', '56', '47', '14', '51', '31', '33', '41', '58', '54', '49', '1', '50', '44', '13', '30', '45', '57', '38', '55', '20', '43', '18', '25', '12', '46', '27', '40', '16', '22', '37', '34', '53', '6', '32', '23', '52', '29', '19', '48', '17', '4', '15', '5', '9', '28']


In [ ]:
import os
import shutil
import random
import pandas as pd
from sklearn.model_selection import train_test_split

dataset_dir = "/root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1/Indian-Traffic Sign-Dataset/Images"
csv_path = "/root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1/Indian-Traffic Sign-Dataset/traffic_sign.csv"
output_dir = "/root/yolo_traffic_sign"
split_ratios = {"train": 0.7, "val": 0.2, "test": 0.1}

os.makedirs(output_dir, exist_ok=True)
for subset in ["train", "val", "test"]:
    for subfolder in ["images", "labels"]:
        os.makedirs(os.path.join(output_dir, subset, subfolder), exist_ok=True)

class_mapping = pd.read_csv(csv_path).set_index("ClassId").to_dict()["Name"]

image_paths = []
labels = []

for class_id in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_id)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image_paths.append(image_path)
            labels.append(int(class_id))

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=split_ratios["test"], random_state=42, stratify=labels
)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=split_ratios["val"] / (1 - split_ratios["test"]),
    random_state=42, stratify=train_labels
)

def create_yolo_label(image_path, label, output_label_dir):
    class_id = label
    yolo_label = f"{class_id} 0.5 0.5 1.0 1.0\n"
    label_name = os.path.splitext(os.path.basename(image_path))[0] + ".txt"
    label_path = os.path.join(output_label_dir, label_name)
    with open(label_path, "w") as f:
        f.write(yolo_label)

def process_subset(image_paths, labels, subset_name):
    for image_path, label in zip(image_paths, labels):
        subset_image_dir = os.path.join(output_dir, subset_name, "images")
        subset_label_dir = os.path.join(output_dir, subset_name, "labels")
        shutil.copy(image_path, subset_image_dir)
        create_yolo_label(image_path, label, subset_label_dir)

process_subset(train_paths, train_labels, "train")
process_subset(val_paths, val_labels, "val")
process_subset(test_paths, test_labels, "test")

classes = sorted(list(class_mapping.keys()))
class_names = [class_mapping[c] for c in classes]
yaml_content = f"""\
train: {os.path.join(output_dir, 'train/images')}
val: {os.path.join(output_dir, 'val/images')}
test: {os.path.join(output_dir, 'test/images')}

nc: {len(classes)}
names: {class_names}
"""

yaml_path = os.path.join(output_dir, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"Dataset prepared successfully in: {output_dir}")
print(f"data.yaml created at: {yaml_path}")

Dataset prepared successfully in: /root/yolo_traffic_sign
data.yaml created at: /root/yolo_traffic_sign/data.yaml


In [ ]:
import yaml

yaml_path = "/root/yolo_traffic_sign/data.yaml"

with open(yaml_path, "r") as file:
    data = yaml.safe_load(file)

print("Train path:", data['train'])
print("Validation path:", data['val'])
print("Test path:", data['test'])
print("Number of classes:", data['nc'])
print("Class names:", data['names'])

Train path: /root/yolo_traffic_sign/train/images
Validation path: /root/yolo_traffic_sign/val/images
Test path: /root/yolo_traffic_sign/test/images
Number of classes: 59
Class names: ['Give way', 'No entry', 'One-way traffic', 'One-way traffic', 'No vehicles in both directions', 'No entry for cycles', 'No entry for goods vehicles', 'No entry for pedestrians', 'No entry for bullock carts', 'No entry for hand carts', 'No entry for motor vehicles', 'Height limit', 'Weight limit', 'Axle weight limit', 'Length limit', 'No left turn', 'No right turn', 'No overtaking', 'Maximum speed limit (90 km/h)', 'Maximum speed limit (110 km/h)', 'Horn prohibited', 'No parking', 'No stopping', 'Turn left', 'Turn right', 'Steep descent', 'Steep ascent', 'Narrow road', 'Narrow bridge', 'Unprotected quay', 'Road hump', 'Dip', 'Loose gravel', 'Falling rocks', 'Cattle', 'Crossroads', 'Side road junction', 'Side road junction', 'Oblique side road junction', 'Oblique side road junction', 'T-junction', 'Y-juncti

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
epochs = 50
patience = 5
imgsz = 640
batch_size = 16
project_name = 'runs/train'
experiment_name = 'traffic_sign_detection'
dataset_folder = '/root/yolo_traffic_sign'

results = model.train(
    data=yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    project=project_name,
    name=experiment_name,
    exist_ok=True,
    patience=patience,
    verbose=True
)
save_dir = model.trainer.save_dir
print(f"The best model is saved at: {save_dir}/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/root/yolo_traffic_sign/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=traffic_sign_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 21.6MB/s]


Overriding model.yaml nc=80 with nc=59

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

      33/50      2.34G     0.0406     0.7114     0.8917         10        640: 100%|██████████| 612/612 [02:54<00:00,  3.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:23<00:00,  3.67it/s]

                   all       2794       2794      0.891      0.954      0.911      0.911


EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 28, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

33 epochs completed in 1.868 hours.
Optimizer stripped from runs/train/traffic_sign_detection/weights/last.pt, 6.3MB
Optimizer stripped from runs/train/traffic_sign_detection/weights/best.pt, 6.3MB

Validating runs/train/traffic_sign_detection/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,017,153 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:23<00:00,  3.82it/s]


                   all       2794       2794      0.888      0.954      0.913      0.913
              Give way         40         40      0.967       0.95      0.964      0.964
              No entry         40         40      0.912      0.975      0.978      0.978
       One-way traffic         40         40      0.762       0.95      0.952      0.952
       One-way traffic         40         40      0.786        0.9      0.945      0.945
No vehicles in both directions         40         40      0.952      0.975      0.994      0.994
   No entry for cycles         64         64      0.989      0.906      0.968      0.968
No entry for goods vehicles         40         40      0.966          1      0.995      0.995
No entry for pedestrians         41         41      0.963          1      0.995      0.995
No entry for bullock carts        120        120      0.985      0.992      0.995      0.995
No entry for hand carts         40         40      0.926          1      0.995      0.995
N

In [ ]:
from google.colab import files

model_path = 'runs/train/traffic_sign_detection/weights/best.pt'

files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
from ultralytics import YOLO

model = YOLO('classification.pt')

classes = ['Give way', 'No entry', 'One-way traffic', 'One-way traffic', 'No vehicles in both directions', 'No entry for cycles', 'No entry for goods vehicles', 'No entry for pedestrians', 'No entry for bullock carts', 'No entry for hand carts', 'No entry for motor vehicles', 'Height limit', 'Weight limit', 'Axle weight limit', 'Length limit', 'No left turn', 'No right turn', 'No overtaking', 'Maximum speed limit (90 km/h)', 'Maximum speed limit (110 km/h)', 'Horn prohibited', 'No parking', 'No stopping', 'Turn left', 'Turn right', 'Steep descent', 'Steep ascent', 'Narrow road', 'Narrow bridge', 'Unprotected quay', 'Road hump', 'Dip', 'Loose gravel', 'Falling rocks', 'Cattle', 'Crossroads', 'Side road junction', 'Side road junction', 'Oblique side road junction', 'Oblique side road junction', 'T-junction', 'Y-junction', 'Staggered side road junction', 'Staggered side road junction', 'Roundabout', 'Guarded level crossing ahead', 'Unguarded level crossing ahead', 'Level crossing countdown marker', 'Level crossing countdown marker', 'Level crossing countdown marker', 'Level crossing countdown marker', 'Parking', 'Bus stop', 'First aid post', 'Telephone', 'Filling station', 'Hotel', 'Restaurant', 'Refreshments']

img_path = '22_640.png'

results = model.predict(img_path)

detected_classes = []
for detection in results[0].boxes:
    detected_class_index = int(detection.cls.item())
    detected_classes.append(detected_class_index)

print(f"Detected class indices: ")
for i in detected_classes:
  print(classes[i])


image 1/1 /content/22_640.png: 640x640 1 Height limit, 337.7ms
Speed: 17.7ms preprocess, 337.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Detected class indices: 
Height limit
